In [5]:
import os
from PIL import Image
import torch
from transformers import DetrConfig, DetrForObjectDetection, DetrFeatureExtractor, Trainer, TrainingArguments
from datasets import load_dataset

In [6]:
num_labels = 41

config = DetrConfig(num_labels=num_labels)
model = DetrForObjectDetection(config)



In [ ]:
feature_extractor = DetrFeatureExtractor()

dataset = load_dataset("json", data_files={"train": "your_file.json"}, field="images")


Generating train split: 0 examples [00:00, ? examples/s]

Failed to load JSON from file '/home/deonx/code/Intel/train_output.json' with error <class 'pyarrow.lib.ArrowInvalid'>: JSON parse error: Column() changed from object to string in row 0


DatasetGenerationError: An error occurred while generating the dataset